# Netflix Movies and TV Shows Analysis

## Description

This dataset contains...

## Loading the data

In [36]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


## Summary and Descriptive Statistics

As we can see from the extract we just printed, we have a mix of numerical and categorical data. So, if we want to use `describe`
to get some summary information and descriptive statistic, it's best to operate on numerical and categorical variables separately.

### Statistics for Numerical Variables

In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
show_id,6234.0,7.670368e+07,1.094296e+07,247747.0,80035801.75,80163367.0,80244888.75,81235729.0
release_year,6234.0,2.013359e+03,8.811620e+00,1925.0,2013.00,2016.0,2018.00,2020.0


### Statistic for Categorical Variables

In [4]:
df.describe(include=[object]).T

,count,unique,top,freq
type,6234,2,Movie,4265
title,6234,6172,Oh My Ghost,3
director,4265,3301,"Raúl Campos, Jan Suter",18
cast,5664,5469,David Attenborough,18
country,5758,554,United States,2032
date_added,6223,1524,"January 1, 2020",122
rating,6224,14,TV-MA,2027
duration,6234,201,1 Season,1321
listed_in,6234,461,Documentaries,299
description,6234,6226,A surly septuagenarian gets another chance at ...,3


### Additional Information

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       6234 non-null   int64 
 1   type          6234 non-null   object
 2   title         6234 non-null   object
 3   director      4265 non-null   object
 4   cast          5664 non-null   object
 5   country       5758 non-null   object
 6   date_added    6223 non-null   object
 7   release_year  6234 non-null   int64 
 8   rating        6224 non-null   object
 9   duration      6234 non-null   object
 10  listed_in     6234 non-null   object
 11  description   6234 non-null   object
dtypes: int64(2), object(10)
memory usage: 584.6+ KB


Given that `describe` doesn't calculate the number of unique values for numerical columns, we can print it ourselves:

In [6]:
df.nunique()

show_id         6234
type               2
title           6172
director        3301
cast            5469
country          554
date_added      1524
release_year      72
rating            14
duration         201
listed_in        461
description     6226
dtype: int64

## Data Transformation
### Missing Data Analysis

We begin our analysis by determining the number of rows with holes for each column:

In [7]:
df.isna().sum()

show_id            0
type               0
title              0
director        1969
cast             570
country          476
date_added        11
release_year       0
rating            10
duration           0
listed_in          0
description        0
dtype: int64

As we can see, we have missing values for:
* `director`: we won't be using it, so we'll discard it.
* `cast`: we won't be using it, so we'll discard it.
* `country`: we'll fill missing values here with the `mode`.
* `date_added`: if you look closely, you'll notice that it's safe to do a `ffill`. Plus, the rows with missing values for
this column are the last in the dataset.
* `rating`: given that there are only 10 empty values. I decided to google the corresponding ratings.

In [8]:
# Dropping cast and director.
df.drop(['director', 'cast'], axis=1, inplace=True)
# Filling the country column with the mode
df['country'].fillna(df['country'].mode()[0], inplace=True)
# Filling the date_added with forward propagation
df['date_added'] = df['date_added'].ffill()

In [9]:
replacements = {
    211: 'TV-MA',
    2411: 'PG-13',
    3288: 'TV-MA',
    4056: 'TV-MA',
    4402: 'TV-G',
    4403: 'TV-G',
    4706: 'TV-14',
    5015: 'TV-MA',
    5234: 'TV-MA',
    6231: 'TV-Y'
}

for show_id, rating in replacements.items():
    df.iloc[show_id, 6] = rating

We can check for missing data once more to confirm we no longer have empty values.

In [10]:
df.isna().sum()

show_id         0
type            0
title           0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

### Dropping additional columns

We now proceed to drop some other columns that we are not going to need for our process.

In [11]:
df.drop(['listed_in', 'description'], axis=1, inplace=True)

### Using `show_id` as index
As the `show_id` column doesn't include duplicates, we can well use it as the index of the `DataFrame`.

In [12]:
df.set_index('show_id', inplace=True)

### Converting values for `date_added` to Dates
If you take a look at the content of the `date_added` column you'll see that it contains dates in a variety
of formats. Lucky for us, `Pandas` can deal create Datetime objects out of all of them:

In [ ]:
df['date_added'] = pd.to_datetime(df['date_added'])

### Converting `type` into a Categorical type
This column contains two possible values. Although it's not really needed for our analysis, we can transform this column
into a categorical one.

In [ ]:
df['type'] = pd.Categorical(df['type'])

### Working with `country` (segregating american content)

If we explore the `country` column, we'll see that it has 544 different values:

In [23]:
df['country'].nunique()

554

We will also notice that roughly half of the rows contain the string "United States" in `country`.

In [34]:
len(df[df['country'].str.contains('United States')].index)

3086

Therefore, we can add a boolean column `american` to flag content produced, at least partially, by America.

In [ ]:
df['american'] = df.apply(lambda row: 'United States' in row['country'], axis=1)

### Extracting `month` and `year` from `date_added`

Having parsed `date_added` to Datetime, we can create additional columns for the `month` and `year` to allow for additional
analysis in a convinient way.

In [ ]:
df['month_added'] = pd.DatetimeIndex(df['date_added']).month
df['year_added'] = pd.DatetimeIndex(df['date_added']).year

### Working with `rating`
If we take a look at the `ratings` columns, we'll see that it contains 14 different values. They are:

In [20]:
df['rating'].unique()

array(['TV-PG', 'TV-MA', 'TV-Y7-FV', 'TV-Y7', 'TV-14', 'R', 'TV-Y', 'NR',
       'PG-13', 'TV-G', 'PG', 'G', 'UR', 'NC-17'], dtype=object)

As you may know, these _maturity ratings_ are earned based on the kind of content of the show, and they stipulate the recommended age for viewers.
Based on the information found on the [Amazon's Help & Customer Service](https://www.amazon.com/gp/help/customer/display.html?nodeId=G2C2CPZWGZWHZ42J) site,
we can map ratings like this:

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Rating</th>
      <th>Age Restriction</th>
    </tr>
  </thead>
  <tbody>
<tr><td>TV-PG</td><td>7</td></tr>
<tr><td>TV-MA</td><td>18</td></tr>
<tr><td>TV-Y7-FV</td><td>7</td></tr>
<tr><td>TV-Y7</td><td>7</td></tr>
<tr><td>TV-14</td><td>16</td></tr>
<tr><td>R</td><td>18</td></tr>
<tr><td>TV-Y</td><td>0</td></tr>
<tr><td>NR</td><td>18</td></tr>
<tr><td>PG-13</td><td>13</td></tr>
<tr><td>TV-G</td><td>0</td></tr>
<tr><td>PG</td><td>7</td></tr>
<tr><td>G</td><td>0</td></tr>
<tr><td>UR</td><td>18</td></tr>
<tr><td>NC-17</td><td>18</td></tr>
  </tbody>
</table>
</div>

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Name</th>
      <th>Age Restriction</th>
    </tr>
  </thead>
  <tbody>
<tr><td>Kids</td><td>0</td></tr>
<tr><td>Older Kids</td><td>7</td></tr>
<tr><td>Teens</td><td>13</td></tr>
<tr><td>Young Adults</td><td>16</td></tr>
<tr><td>Adults</td><td>18</td></tr>
  </tbody>
</table>
</div>

We can create additional `Series` for this information that we won't join just yet.

In [41]:
ratings_data = { 'TV-PG': 7,'TV-MA': 18,'TV-Y7-FV': 7,'TV-Y7': 7,
            'TV-14': 16,'R': 18,'TV-Y': 0,'NR': 18,'PG-13': 13,
            'TV-G': 0,'PG': 7,'G': 0,'UR': 18,'NC-17': 18}

ratings = pd.Series(ratings_data)
ratings

TV-PG        7
TV-MA       18
TV-Y7-FV     7
TV-Y7        7
TV-14       16
R           18
TV-Y         0
NR          18
PG-13       13
TV-G         0
PG           7
G            0
UR          18
NC-17       18
dtype: int64

In [42]:
ages_data = {0: 'Kids', 7: 'Older Kids', 13: 'Teens', 16: 'Young Adults', 18: 'Adults'}
ages = pd.Series(ages_data)
ages

0             Kids
7       Older Kids
13           Teens
16    Young Adults
18          Adults
dtype: object